##Adam Kritz Bach Project

Welcome to my Bach Project. I know absolutely nothing about notes and chords. I played violin for 1 year in first grade, as it was a required course in my private school, and I absolutely despised it. From that year on I was permanently scarred, and I haven't touched a musical instrument since.

Luckily, I do not need to actually know about music to make a classifier. The data could be about anything, and I could still make a classifier for it. Which is exactly what I intend to do. 

Contents:

1. Get that file in
2. Explore the data for a bit
3. Split the data
4. KNN classifier
5. Naive Bayes Classifer
6. Decision Tree classifier
7. Multi-Layer Perceptron Classifier
8. Random Forest Classifier
9. Conclusion

#Read File

Here I import the file and take a look at the first few rows to see what I am working with. There is nothing really of note to see here, as I already know the columns I will be running the classifier on from the directions. 

In [4]:
import pandas as pd
!wget 'https://raw.githubusercontent.com/zacharski/ml-class/master/data/bach.zip' 
!unzip 'bach.zip'

--2021-02-16 05:12:35--  https://raw.githubusercontent.com/zacharski/ml-class/master/data/bach.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41761 (41K) [application/zip]
Saving to: ‘bach.zip’

bach.zip            100%[===================>]  40.78K  --.-KB/s    in 0.002s  

2021-02-16 05:12:35 (20.8 MB/s) - ‘bach.zip’ saved [41761/41761]

Archive:  bach.zip
  inflating: bach.csv                


In [5]:
bach = pd.read_csv('bach.csv')

In [6]:
bach.head()

,choral_ID,event_number,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter,chord_label
0,000106b_,1,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
1,000106b_,2,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,5,C_M
2,000106b_,3,YES,NO,NO,NO,YES,NO,NO,YES,NO,NO,NO,NO,E,2,C_M
3,000106b_,4,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,3,F_M
4,000106b_,5,YES,NO,NO,NO,NO,YES,NO,NO,NO,YES,NO,NO,F,2,F_M


#Explore

I usually don't do a lot of exploration right off the bat when looking at data sets, instead I just work until I encounter a problem, then I go back and fix it. Otherwise it is like looking for a needle in a haystack. 

Here I double check to see if there are any columns I missed. There are not in this case, but I have missed columns before so it is always good to check.

In [7]:
bach.columns

Index(['choral_ID', 'event_number', 'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G',
       'G#', 'A', 'A#', 'B', 'bass', 'meter', 'chord_label'],
      dtype='object')

Here I look at a description of the data. The main row to note here is the "unique" row, as it lets us know how many different options there are in each row. All the different notes only have 2 options, either they are being played or they are not being played, while the "bass" column has 16 possible options. In this case I noticed the "meter" column is numerical data, while every other column is in categorical data. 

The "frequency" row is also cool, as it shows which notes are played the most and least often overall. G# is played the most overall, while A is played the least.

In [8]:
bach.describe(include="all")

,choral_ID,event_number,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter,chord_label
count,5665,5665.000000,5665,5665,5665,5665,5665,5665,5665,5665,5665,5665,5665,5665,5665,5665.000000,5665
unique,60,NaN,2,2,2,2,2,2,2,2,2,2,2,2,16,NaN,102
top,002908ch,NaN,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,D,NaN,D_M
freq,207,NaN,3875,4711,3300,4956,3540,4381,4253,3523,5006,3290,4644,3874,689,NaN,503
mean,NaN,53.374404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.134863,NaN
std,NaN,37.268208,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.109710,NaN
min,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN
25%,NaN,24.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,NaN
50%,NaN,48.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,NaN
75%,NaN,75.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,NaN


Here I checked to see if there are an NaN values. There are not. Yay.

In [9]:
 bach.isnull().values.any()

False

#Split the Data

I one hot encoded all the columns. This is quite easy for all the note columns, and I just manually changed YES to 1, and NO to 0. The bass column requires a whole bunch of new columns, as it has 16 unique values.

In [10]:
bach = bach.replace(['YES'], 1)
bach = bach.replace(['NO'], 0)

one_hot = pd.get_dummies(bach['bass'], prefix= 'bass')
bachs = bach.drop('bass', axis=1)
bach = bach.join(one_hot)

bach.head()

,choral_ID,event_number,C,C#,D,D#,E,F,F#,G,G#,A,A#,B,bass,meter,chord_label,bass_A,bass_A#,bass_Ab,bass_B,bass_Bb,bass_C,bass_C#,bass_D,bass_D#,bass_Db,bass_E,bass_Eb,bass_F,bass_F#,bass_G,bass_G#
0,000106b_,1,1,0,0,0,0,1,0,0,0,1,0,0,F,3,F_M,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,000106b_,2,1,0,0,0,1,0,0,1,0,0,0,0,E,5,C_M,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,000106b_,3,1,0,0,0,1,0,0,1,0,0,0,0,E,2,C_M,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,000106b_,4,1,0,0,0,0,1,0,0,0,1,0,0,F,3,F_M,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,000106b_,5,1,0,0,0,0,1,0,0,0,1,0,0,F,2,F_M,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


Here I split the data with the code from the directions.

In [11]:
bach.set_index('choral_ID', inplace=True)
bFeatures = bach.drop('chord_label', axis=1)
bLabels = bach['chord_label']
from sklearn.model_selection import train_test_split
bach_train_features, bach_test_features, bach_train_labels, bach_test_labels = train_test_split(bFeatures, bLabels, test_size = 0.2, random_state=42)

Lastly, I made a "features" list, so I do not have to worry about retyping the full thing every time I need to use it.

In [12]:
features = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B', 'meter', 'bass_A',	'bass_A#',	'bass_Ab', 'bass_B',	
            'bass_Bb',	'bass_C',	'bass_C#', 'bass_D', 'bass_D#', 'bass_Db',	'bass_E',	'bass_Eb',	'bass_F',	'bass_F#',	'bass_G',	'bass_G#']

#KNN

Personally, I am not a big fan of KNN classifiers. I like to refer to it as "baby's first classifier" as it is easy to understand, but often not very good. I will try it out anyway of course, but I do not predict a very high accuracy. 

After a little messing around, the highest accuracy seems to be with 6 neighbors.

In [13]:
from sklearn.neighbors import KNeighborsClassifier
knnbach = KNeighborsClassifier(n_neighbors=6)
knnbach.fit(bach_train_features[features], bach_train_labels)
knnbachpredict = knnbach.predict(bach_test_features[features])

from sklearn.metrics import accuracy_score
knnscore = accuracy_score(knnbachpredict, bach_test_labels)
knnscore

0.7131509267431597

Here I tried changing the weights to distance, instead of uniform. This means that the closer a neighbor is, the higher influence it has. This boosted the accuracy up a bit more.

In [14]:
knnbach = KNeighborsClassifier(n_neighbors=8, weights='distance')
knnbach.fit(bach_train_features[features], bach_train_labels)
knnbachpredict = knnbach.predict(bach_test_features[features])

knnscore = accuracy_score(knnbachpredict, bach_test_labels)
knnscore

0.733451015004413

Changing the "algorithm" hyperparameter or p-value did not seem to help much, so this is about as far as KNN can go without testing all the hyperparameters. 73% is not a bad start at all for KNN. 

#Naive Bayes

I am a massive fan of Naive Bayes classifiers, so I have really high hopes for this one. 

In [15]:
from sklearn.naive_bayes import GaussianNB
naivebach = GaussianNB()
naivebach.fit(bach_train_features[features], bach_train_labels)
naivebachpredict = naivebach.predict(bach_test_features[features])

naivescore = accuracy_score(naivebachpredict, bach_test_labels)
naivescore

0.08737864077669903

Well that was disappointing. Let me try to change some hyper parameters.

In [16]:
naivebach = GaussianNB(var_smoothing=0.16)
naivebach.fit(bach_train_features[features], bach_train_labels)
naivebachpredict = naivebach.predict(bach_test_features[features])

naivescore = accuracy_score(naivebachpredict, bach_test_labels)
naivescore

0.6910856134157105

That is better, but still does not beat KNN. Maybe it will work better with a Multinomial Naive Bayes Classifier?

In [17]:
from sklearn.naive_bayes import MultinomialNB
naivebach = MultinomialNB()
naivebach.fit(bach_train_features[features], bach_train_labels)
naivebachpredict = naivebach.predict(bach_test_features[features])

naivescore = accuracy_score(naivebachpredict, bach_test_labels)
naivescore

0.6875551632833187

I am disappointed :(

The Naive Bayes classifier caps out around 69% from what I am seeing. It seems as though my favorite classifier is not the best for this.

#Decision Tree

This is a basic decision tree classifier.

In [18]:
from sklearn import tree
dtbach = tree.DecisionTreeClassifier(criterion='entropy')
dtbach.fit(bach_train_features[features], bach_train_labels)
dtbachpredict = dtbach.predict(bach_test_features[features])

dtscore = accuracy_score(dtbachpredict, bach_test_labels)
dtscore

0.7175639894086496

Here I tried the gini as the citerion, which has something to do with Gini impurity. Overall, it worked just slightly better.

In [19]:
dtbach = tree.DecisionTreeClassifier(criterion='gini')
dtbach.fit(bach_train_features[features], bach_train_labels)
dtbachpredict = dtbach.predict(bach_test_features[features])

dtscore = accuracy_score(dtbachpredict, bach_test_labels)
dtscore

0.7219770520741394

Surprisingly, I could not get much higher than 72% for a decision tree classifier, no matter what I changed. 

#MLP Classifier

I will now try out some other classifiers I found online.

In [20]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.neural_network import MLPClassifier
MLPbach = MLPClassifier()
MLPbach.fit(bach_train_features[features], bach_train_labels)
MLPbachpredict = MLPbach.predict(bach_test_features[features])

MLPscore = accuracy_score(MLPbachpredict, bach_test_labels)
MLPscore

0.735216240070609

74% is pretty high right off the bat, wow!

However, I tried changing some more hyper parameters around and not much changed, so I will stick with th 74% accuracy for now. 

#Random Forest Classifier

This is another one of my favortie classifiers, I learned about it in my business analytics class.

In [21]:
from sklearn.ensemble import RandomForestClassifier
RFbach = RandomForestClassifier()
RFbach.fit(bach_train_features[features], bach_train_labels)
RFbachpredict = RFbach.predict(bach_test_features[features])

RFscore = accuracy_score(RFbachpredict, bach_test_labels)
RFscore

0.7210944395410415

Changing up some hyperparameters...

In [22]:
RFbach = RandomForestClassifier(criterion='entropy', max_depth=10)
RFbach.fit(bach_train_features[features], bach_train_labels)
RFbachpredict = RFbach.predict(bach_test_features[features])

RFscore = accuracy_score(RFbachpredict, bach_test_labels)
RFscore

0.7493380406001765

This hit the highest so far, with 75%+ on some runs.

Because Random Forest is doing so well, I will try it out with a full hyper paramter grid. I mainy used hyperparamters I knew actually affected the data, as many of them seemed to do nothing in this case.

In [ ]:
#this took about an hour to process, so I included the results in the section so you do not have to run it. 
from sklearn.model_selection import GridSearchCV
hyperparam_grid = [{'criterion': ['gini', 'entropy'],
    'max_depth': [5,10,15, None],
    'min_samples_split': [1, 2, 5],
    'min_samples_leaf': [0.1, 1, 2],
    'max_features':['auto', 'sqrt', 'log2'],
    'max_leaf_nodes':[None, 5],
    'n_jobs': [-1, None, 1]}]
gridsearch = GridSearchCV(RFbach, hyperparam_grid, cv=10)
gridsearch.fit(bach_train_features[features], bach_train_labels)

In [ ]:
print(gridsearch.best_params_)

In [ ]:
RFbach = RandomForestClassifier(criterion='entropy', max_depth= 10, max_features = 'auto', max_leaf_nodes=None, min_samples_leaf = 1, min_samples_split = 2, n_jobs = None)
RFbach.fit(bach_train_features[features], bach_train_labels)
RFbachpredict = RFbach.predict(bach_test_features[features])

RFscore = accuracy_score(RFbachpredict, bach_test_labels)
RFscore

It turns out that these are all the default settings, meaning that this classifier is likely already optimized. It still scores around 75% accuracy.

#Conclusion

I tried a couple other classifiers, but Random Forest was better than all of them, so I chose not to include them. 

Clearly I should not have dismissed KNN, as it is one of the better ones in this case. Looking around online, KNN does fairly well against other classifiers, and generally seems to do consistently well. 

In the end, it seems like Random Forest is the best in this case. The directions say I should try to get to 85% accuracy, but I have no idea how I would even get that close.

Sources: 

List of classifiers: https://scikit-learn.org/stable/supervised_learning.html

Classifier comparison: https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html